## 1. Instalamos dependencias y librerias.

In [ ]:
%pip install -U -q langchain>=0.2.5 langchain-community>=0.2.0 langchain-text-splitters>=0.2.0 langchain-google-genai>=0.0.10 chromadb>=0.5.0 tiktoken>=0.7.0 pypdf>=4 python-dotenv>=1.0.1

In [ ]:
import os
from pathlib import Path
from typing import List

from dotenv import load_dotenv
load_dotenv()  # Carga variables de entorno (GOOGLE_API_KEY, etc.)

# LangChain loaders, splitters, vectorstore, LLM/embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Embeddings y modelo de chat con Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Utilidad
from tqdm import tqdm

## 2. Configuramos el entorno

In [ ]:
# 🛠️ Configuración
PDF_DIR = Path("./docs")             # <- Cambia a tu carpeta con PDFs
PERSIST_DIR = Path("./chroma_pdfs")  # Carpeta donde se guardará Chroma
PERSIST_DIR.mkdir(parents=True, exist_ok=True)

CHUNK_SIZE = <FILL_IN>
CHUNK_OVERLAP = <FILL_IN>
TOP_K = 4

# Modelos (ajusta si quieres otros)
EMBEDDING_MODEL = "<FILL_IN>"
CHAT_MODEL = "<FILL_IN>"

# Verificar clave
assert os.getenv("GOOGLE_API_KEY"), "Falta GOOGLE_API_KEY en variables de entorno o .env"
print(f"📁 Carpeta PDFs: {PDF_DIR.resolve()}")
print(f"🗂️ Persistencia Chroma: {PERSIST_DIR.resolve()}")

## 3. Cargamos los pdfs

In [ ]:
def load_pdfs_from_dir(directory: Path, recursive: bool = True):
    pattern = "**/*.pdf" if recursive else "*.pdf"
    pdf_paths = sorted([p for p in directory.glob(pattern) if p.is_file()])
    all_docs = []
    for pdf in tqdm(pdf_paths, desc="Cargando PDFs"):
        try:
            docs = PyPDFLoader(str(pdf)).load()
            # Añadimos metadatos mínimos útiles
            for d in docs:
                d.metadata = d.metadata or {}
                d.metadata["source"] = str(pdf.resolve())
            all_docs.extend(docs)
        except Exception as e:
            print(f"⚠️ Error leyendo {pdf}: {e}")
    print(f"📚 Documentos (páginas) cargados: {len(all_docs)}")
    return all_docs

raw_docs = load_pdfs_from_dir(PDF_DIR, recursive=True)

## 4. Chunking

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=<FILL_IN>,
    chunk_overlap=<FILL_IN>,
    separators=["\n\n", "\n", " ", ""],
)
chunks = splitter.split_documents(raw_docs)
print(f" Chunks generados: {len(chunks)}")
chunks[:2]  # vista rápida

## 5. Embeddings

In [ ]:
# embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
embeddings = GoogleGenerativeAIEmbeddings(model=<FILL_IN>)
vectorstore = Chroma.from_documents(
    documents=<FILL_IN>,
    embedding=<FILL_IN>,
    persist_directory=str(PERSIST_DIR),
)
retriever =<FILL_IN>
print(" Chroma persistido")

## 6. Creamos la Tool (Retriever)

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    <FILL_IN>,
    "<FILL_IN>",#nombre
    "<FILL_IN>",#Descripcion
)

3 Test the tool

In [ ]:
retriever_tool.invoke({"query": "<FILL_IN>"})

3. Generamos la query

In [ ]:
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model
from langchain_google_genai import ChatGoogleGenerativeAI

response_model=  ChatGoogleGenerativeAI(model=<FILL_IN>, temperature=<FILL_IN>)

def genera_query_o_responde(state: MessagesState):
    """Llama al modelo para generar una respuesta basada en el estado actual.
      Dada la pregunta, decidirá si recupera información usando la herramienta de recuperación o simplemente responde al usuario."""
   
    response = (
        response_model
        .bind_tools([<FILL_IN>]).invoke(state[<FILL_IN>])
    )
    return {"messages": [response]}

3.1 Lo probamos con una pregunta aleatoria

In [ ]:
input = {"messages": [{"role": "user", "content": "<FILL_IN>"}]}
respuesta= genera_query_o_responde(input)
print (respuesta)

In [ ]:
#cogemos el ultimo mensaje
respuesta["messages"][-1]

In [ ]:
respuesta["messages"][-1].pretty_print()

3.2  Hacer una pregunta que requiera una busqueda semantica

In [ ]:
input = {
    "messages": [
        {
            "role": "user",
            "content": "<FILL_IN>",
        }
    ]
}
genera_query_o_responde(input)["messages"][-1].pretty_print()

4. Vemos la relevancia del documento

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

GRADE_PROMPT = (
    "Eres un evaluador que determina la relevancia de un documento recuperado respecto a una pregunta del usuario. \n "
    "Aquí tienes el documento recuperado: \n\n {context} \n\n"
    "Aquí tienes la pregunta del usuario: {question} \n"
    "Si el documento contiene palabra(s) clave o significado semántico relacionado con la pregunta del usuario, califícalo como relevante. \n"
    "Da una puntuación binaria 'si' o 'no' para indicar si el documento es relevante para la pregunta."

)


class GradeDocuments(BaseModel):
    """Califica los documentos utilizando una puntuación binaria para comprobar su relevancia"""

    binary_score: str = Field(
        description="Puntuación : 'si' si es relevante, o 'no' si no lo es"
    )


grader_model = ChatGoogleGenerativeAI(model=<FILL_IN>, temperature=<FILL_IN>)


def grade_documents(
    state: MessagesState,
) -> Literal["genera_respuesta", "rescribir_question"]:
    print("grade_documents")
    """Determina si los documentos recuperados son relevantes para la pregunta."""
    question = <FILL_IN>
    context = <FILL_IN>
    
    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score
    print ("score :", score)
    if score == "si":
        return "genera_respuesta"
    else:
        return "rescribir_question"

4.1 Ejecutar con una respuesta irrelevante

In [ ]:
#simulamos la respuesta de la tool mediante mensajes
from langchain_core.messages import convert_to_messages

input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "<FILL_IN>",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "<FILL_IN>"},
                    }
                ],
            },
            {"role": "tool", "content": "<FILL_IN>", "tool_call_id": "1"},
        ]
    )
}


grade_documents(input)

4.2 Coomprobar que el documento/respuesta relevante lo clasifica como tal

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "<FILL_IN>",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "<FILL_IN>"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "<FILL_IN>",
                "tool_call_id": "1",
            },
        ]
    )
}
grade_documents(input)

5 Rescribir la pregunta

5.1 Crear el nodo de reesctirura de la pregunta

In [ ]:
REWRITE_PROMPT = (
    "Analiza detenidamente la siguiente pregunta e intenta comprender la intención o el significado profundo que transmite.\n"
    "Pregunta original:"
    "\n ------- \n"
    "{question}"
    "\n ------- \n"
    "Ahora, reescribe la pregunta para que sea más clara, precisa y fácil de entender:"
)


def rescribir_question(state: MessagesState):
    """Rescribe/Mejora la pregunta original del usuario."""
    messages = state["messages"]
    question = <FILL_IN>
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [{"role": "user", "content": response.content}]}

5.2 Probamos  rescribir la pregunta

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "<FILL_IN>",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "<FILL_IN>"},
                    }
                ],
            },
            {"role": "tool", "content": "<FILL_IN>", "tool_call_id": "1"},
        ]
    )
}

response = rescribir_question(input)
print(response["messages"][-1]["content"])

## 6. Generamos la Respuesta.

### 6.1. Construimos el nodo generate_answer.
Si superamos las comprobaciones del evaluador (grader), podemos generar la respuesta final basándonos en la pregunta original y el contexto recuperado

In [ ]:
GENERATE_PROMPT = (
    "Eres un asistente para tareas de preguntas y respuestas. "
    "Utiliza los siguientes fragmentos de contexto recuperado para responder a la pregunta. "
    "Si no sabes la respuesta, simplemente indica que no la sabes. "
    "Utiliza un máximo de tres frases y mantén la respuesta concisa.\n"
    "Pregunta: {question} \n"
    "Contexto: {context}"
)



def genera_respuesta(state: MessagesState):
    """Genera la respuesta."""
    print("genera_respuesta")
    question = <FILL_IN>
    context = <FILL_IN>
    prompt = GENERATE_PROMPT.format(question=question, context=context)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    return {"messages": [response]}

### 6.2. Try it out.

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "<FILL_IN>",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "<FILL_IN>"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "<FILL_IN>",
                "tool_call_id": "1",
            },
        ]
    )
}

response = genera_respuesta(input)
response["messages"][-1].pretty_print()

## 7. Configurar el grafo.
### 7.1 Importamos los elementos necesarios para construir el grafo

In [ ]:
#Hasta aqui ya tenemos todo el procesamiento y necesitamos 
#importar los elementos necesarios para construir el grafo. Xavi
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

### 7.2 Ensamblamos el workflow
#### 7.2.1 Añadimos los nodos
![grafo_nodos.jpg](attachment:d4922386-7270-40c8-b5a2-113b13c5be37.jpg)

In [ ]:
workflow = StateGraph(<FILL_IN>)
workflow.add_node(<FILL_IN>)
workflow.add_node("retrieve", ToolNode([<FILL_IN>]))
workflow.add_node(<FILL_IN>)
workflow.add_node(<FILL_IN>)

#### 7.2.1 Añadimos las aristas
![grafo_aristas.jpg](attachment:4023440b-77e0-42fa-b06b-15d0c62f5714.jpg)

In [ ]:
workflow.add_edge(<FILL_IN>, "genera_query_o_responde")

workflow.add_conditional_edges(
    "genera_query_o_responde",
    # Evalúa la decisión del LLM (llama a la herramienta retriever_tool o responde al usuario)
    tools_condition,
    {
        # Traduce las salidas de la condición a nodos dentro de nuestro grafo
        "tools": "retrieve",
        END: END,
    },
)

#
workflow.add_conditional_edges(
    "retrieve",
    # Evalúa la decisión del agente con la funcion de condicion
    <FILL_IN>,  
)

workflow.add_edge("genera_respuesta", <FILL_IN>)

workflow.add_edge("rescribir_question", "genera_query_o_responde")

#### 7.2.3 Compilamos el Grafo

In [ ]:
graph = <FILL_IN>

#### 7.2.4 Pintamos el Grafo

In [ ]:
from IPython.display import Image, display
display(Image(<FILL_IN>))

#### 7.4 Ver como funciona el grafo.

In [ ]:
from pprint import pprint  # para imprimir bonito

for chunk in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "<FILL_IN>"
            }
        ]
    }
):
    for node, update in chunk.items():
        print(f"📘 Update from node: {node}")
        print("-" * 40)

        messages = update.get("messages", [])
      
        last_msg = messages[-1]

        try:
            if isinstance(last_msg, dict):
                if "content" in last_msg:
                    print("📝 Contenido textual:")
                    print(last_msg["content"])
                elif "tool_calls" in last_msg:
                    print("🔧 Llamada a función:")
                    pprint(last_msg["tool_calls"])
                else:
                    print("🕵️ Mensaje dict sin content/tool_calls:")
                    pprint(last_msg)
            elif hasattr(last_msg, "content"):
                print("📝 Contenido desde objeto:")
                print(last_msg.content)
            else:
                print("🕵️ Mensaje desconocido:")
                pprint(last_msg)

        except Exception as e:
            print("❌ Error leyendo el mensaje:", str(e))
            pprint(last_msg)

        print("-" * 40 + "\n")

#### 7.5 Hacer una ejecución del grafo

In [ ]:
input = {
    "messages": [
        {
            "role": "user",
            "content": "<FILL_IN>",
        }
    ]
}
from IPython.display import display, Markdown

display(Markdown(graph.invoke(input)["messages"][-1].content))